# **DB 생성 및 관리**

## **1.환경준비**

In [1]:
import pandas as pd
import sqlite3
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import emergency as em
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## **2.DB 작업**

### (1) DB 생성 및 연결

* 데이터베이스 연결
    * 폴더는 미리 생성
    * 해당 파일이 없으면 새로 생성, 있으면 연결

In [2]:
path = 'testdb.db'
conn = sqlite3.connect(path)

### (2) DB 작업 절차

* 1) 커서 활용 : 대부분의 SQL 작업 가능
    * ① 연결
    * ② 커서 선언
    * ③ 작업 : sql
    * ④ 커밋(수정작업 저장)
    * ⑤ 연결종료

* 2) pandas 함수 활용 : 주로 insert, update, delete, select, create table & insert
    * ① 연결
    * ② 작업 : pandas 함수 이용
    * ③ 연결종료

#### 1) 테이블 생성
* 테이블 생성 작업
    * DB Browser for SQLite 에서 직접 수행
    * 파이썬에서 **연결**하고, **커서**를 생성해서 작업

In [3]:
# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS test (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

#### 2) insert
* 데이터프레임에서 직접 DB로 보내기 : df.to_sql

In [4]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : to_sql
data = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie', 'David'], 'age': [30, 25, 35, 40]})
data.to_sql('test', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

# ③ 연결 종료
conn.close()
data

,name,age
0,Alice,30
1,Bob,25
2,Charlie,35
3,David,40


#### 3) Select

In [ ]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM test', conn)
display(df)

# ③ 연결 종료
conn.close()

,id,name,age
0,1,Alice,30
1,2,Bob,25
2,3,Charlie,35
3,4,David,40


#### 4) Drop

* 커서로만 작업 가능

In [6]:
# ① 연결
conn = sqlite3.connect(path)

# ② 커서 선언
cursor = conn.cursor()

# ③ 작업 : 테이블 삭제
cursor.execute('DROP TABLE IF EXISTS test')

# ④ 커밋(수정작업 저장)
conn.commit()

# ⑤ 연결종료
conn.close()

## **3.관리용 DB**


### (1) DB 생성

* DB를 아래 경로와 이름으로 생성
    * project7/db/em.db

In [7]:
# path = './db/em.db'
# # 연결결
# conn = sqlite3.connect(path)

# # # ② 작업 : to_sql
# # data = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie', 'David'], 'age': [30, 25, 35, 40]})
# # data.to_sql('test', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성
# # dataz
# # # ③ 연결 종료
# # conn.close()
# # data


### (2) log 테이블 생성
* 칼럼 구성

|column name | descrition | data type | comment |
|----|----|----|----|
|datetime | 요청일시 | text | yyyy-mm-dd hh:mi:ss |
|input_text | 입력text | text |  |
|input_latitude | 입력 위도 | real |  |
|input_longitude | 입력 경도 | real |  |
|em_class | 응급 등급(1~5) | integer |  |
|hospital1 | 추천병원명 | text |  |
|addr1 | 추천병원주소 | text |  |
|tel1 | 추천병원전화번호 | text |  |
|hospital2 | 추천병원명2 | text |  |
|addr2 | 추천병원주소2 | text |  |
|tel2 | 추천병원전화번호2 | text |  |
|hospital3 | 추천병원명3 | text |  |
|addr3 | 추천병원주소3 | text |  |
|tel3 | 추천병원전화번호3 | text |  |

In [ ]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : to_sql
data = pd.DataFrame({'datetime':[],'input_text':[],'input_latitude':[],'input_longitude':[],'em_class':[],'hospital1':[],'addr1':[],'tel1':[],'hospital2':[],'addr2':[],'tel2':[],'hospital3':[],'addr3':[],'tel3':[]})
data.to_sql('test', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

# cursor.execute('''
# INSERT INTO emergency_logs (datetime, input_text, input_latitude, input_longitude, em_class, hospital1, addr1, tel1, hospital2, addr2, tel2, hospital3, addr3, tel3)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
# ''', ('2024-12-18 15:00:00', '응급 상황 발생', 37.5665, 126.9780, 2, '서울병원', '서울특별시 종로구', '02-123-4567'))

conn.commit()
print("데이터가 삽입되었습니다.")

# ③ 연결 종료
conn.close()
data

### (3) log 테이블 입력 함수

* 현재 일시 저장하기 예제

            # 현재 연월일 시분초 뽑기
            from datetime import datetime
            dt = datetime.now()
            dt = dt.strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
import emergency as em
from datetime import datetime
dt = datetime.now()
dt = dt.strftime("%Y/%m/%d, %H:%M:%S")

path = './'
df_emergency = pd.read_csv(path + '응급실 정보.csv')

def pipeline(path, audio, location):
  OpenAIClient = em.LoadOpenAIClient(path)
  audio_path = path + 'audio/'
  audioName = audio + '.mp3'
  a = em.AudioToText(audio_path, audioName, OpenAIClient)
  sumText = em.TextToSummary(a, OpenAIClient)
  model_path = path + 'fine_tuned_bert'
  result = em.predict(sumText,  model_path, 'cpu')
  if(result < 3):
    print('응급상황인것으로 판단됩니다. 응급실로 이동하세요')
  else:
    print('응급상황이 아닌 것으로 판단됩니다. 증상이 악화된다면 응급실로 이동하세요')
  return em.RecommandHospital(location, df_emergency),result, sumText

my_location = (36.339073,127.966817)
df,result,sumText = pipeline(path, 'audio2', my_location)

# 테이블 생성 및 여러 데이터 한 번에 삽입
cursor.execute('''
INSERT INTO emergency_logs (datetime, input_text, input_latitude, input_longitude, em_class, hospital1, addr1, tel1, hospital2, addr2, tel2, hospital3, addr3, tel3)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) 
''', (str(dt),sumText, my_location[0], my_location[1], result, df.loc[0,'병원이름'], df.loc[0,'주소'], df.loc[0,'전화번호 1'], df.loc[1,'병원이름'], df.loc[1,'주소'], df.loc[1,'전화번호 1'],df.loc[2,'병원이름'], df.loc[2,'주소'], df.loc[2,'전화번호 1']))

conn.commit()

print("데이터가 삽입되었습니다.")


응급상황인것으로 판단됩니다. 응급실로 이동하세요
데이터가 삽입되었습니다.
